## 1. KoELECTRA

In [ ]:
!pip install transformers
!pip install torch
!git clone gttp://github.com/ZIZUN/korean-malicious-comments-dataset.git

Cloning into 'korean-malicious-comments-dataset'...
git: 'remote-gttp' is not a git command. See 'git --help'.

The most similar command is
	remote-http


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [ ]:
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:", device)


device: cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_gas = pd.read_csv("/content/drive/MyDrive/NLPicasso/데이터 전처리/가스라이팅 증강.csv")
df_chatbot = pd.read_csv("/content/drive/MyDrive/NLPicasso/데이터 전처리/ChatbotData.csv")
df_gas.columns = ["prompt", "competition",	"label"]
df_chatbot.columns = ["prompt", "competition",	"label"]

df_gas['label'].fillna(1, inplace=True)
df_chatbot['label'] = df_chatbot['label'].replace([1, 2], 0)

df_all = pd.concat([df_gas, df_chatbot], axis=0, ignore_index=True)
df_all.drop(['prompt'], axis=1, inplace=True)
df_all = df_all[df_all['label'] != ',']
df_all['label'] = df_all['label'].astype(int)
df_all.sample(frac=1)

competition  label
6206       혼자 풀릴 때까지 놔둬야하는데 기다리는게 힘들 거예요.      0
2344   쪽팔린 것좀 알아 … 그 나이먹고 경차 탈면 줄이라도 빨라야지      1
13011                    데이트 계획 짜오라고 해봐요.      0
2230                    장난해? 자기 말을 어떻게 믿어      1
7891                         일기예보 확인해보세요.      0
...                                   ...    ...
14679                시간이 좀 걸리겠지만 가능할 거예요.      0
3953                     제가 당신을 좋아하고 있어요.      0
11730                           후련해질 거예요.      0
2802                         여기 많이 와봤자나요?      1
9559                         또 아침이 올 거예요.      0

[15219 rows x 2 columns]

In [ ]:
train_data = df_all.sample(frac=0.8, random_state=42)
test_data = df_all.drop(train_data.index)

In [ ]:
model_name= "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenized_train_sentences = tokenizer(
    list(train_data["competition"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [ ]:
"""print(tokenized_train_sentences[3])
print(tokenized_train_sentences[3].tokens)
print(tokenized_train_sentences[3].ids)
print(tokenized_train_sentences[3].attention_mask)"""


'print(tokenized_train_sentences[3])\nprint(tokenized_train_sentences[3].tokens)\nprint(tokenized_train_sentences[3].ids)\nprint(tokenized_train_sentences[3].attention_mask)'

In [ ]:
# Tokenizing 'prompt' and 'competition' columns simultaneously
tokenized_test_sentences = tokenizer(
    list(train_data["competition"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] =  torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

In [ ]:
train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
!pip install pytorch-accelerated

In [ ]:
!pip install accelerate -U`

/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
!pip install transformers[torch]

In [ ]:
training_args = TrainingArguments(output_dir='./',
                                  num_train_epochs=10,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=64,
                                  logging_dir='./logs',
                                  logging_steps=500, # 이 부분을 수정했습니다.
                                  save_total_limit=2,
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # 이 부분을 수정했습니다.
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:
trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset=test_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-32-84dc683ee71e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


<ipython-input-32-84dc683ee71e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-32-84dc683ee71e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-32-84dc683ee71e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-32-84dc683ee71e>:6: UserWarning: To copy construct from a 

TrainOutput(global_step=3810, training_loss=0.5173776100939653, metrics={'train_runtime': 421.9271, 'train_samples_per_second': 72.145, 'train_steps_per_second': 9.03, 'total_flos': 641353752991200.0, 'train_loss': 0.5173776100939653, 'epoch': 10.0})

In [ ]:
!pip install transformers --upgrade

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)

<ipython-input-32-84dc683ee71e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.43710535764694214,
 'eval_accuracy': 0.8301576872536137,
 'eval_f1': 0.4337349397590362,
 'eval_precision': 0.7764705882352941,
 'eval_recall': 0.3009118541033435,
 'eval_runtime': 7.9357,
 'eval_samples_per_second': 383.584,
 'eval_steps_per_second': 6.049,
 'epoch': 10.0}

In [ ]:
# 0: 가스라이팅x, 1: 가스라이팅
def sentence_predict(sent):
    # 평가모드로 변경
    model.eval()

    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 모델이 위치한 GPU로 이동
    tokenized_sent.to(device)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
            )

    # 결과 return
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    if result == 1:
        result = " >> 가스라이팅 👿"
    elif result == 0:
        result = " >> 정상 😀"
    return result
#0 입력시 종료
while True:
    sentence = input("댓글을 입력해주세요: ")
    if sentence == "0":
        break
    print(sentence_predict(sentence))
    print("\n")


댓글을 입력해주세요: 다 너 잘못이야
 >> 정상 😀


댓글을 입력해주세요: 미안해
 >> 가스라이팅 👿


댓글을 입력해주세요: 안녕
 >> 가스라이팅 👿


댓글을 입력해주세요: 안녕
 >> 가스라이팅 👿


댓글을 입력해주세요: 한국어
 >> 정상 😀


댓글을 입력해주세요: 착각하지마
 >> 정상 😀


댓글을 입력해주세요: 쪽팔린 것좀 알아 … 그 나이먹고 경차 탈면 줄이라도 빨라야지
 >> 가스라이팅 👿


댓글을 입력해주세요: 0
